# Section 3. Hypothesis Testing

### CONTENTS

* <a href='02- DSC 2022 Exploratory Data Analysis.ipynb#top'>**Section 2. Exploratory Data Analysis**</a> 
* <a href='03 - DSC 2022 Hypothesis testing.ipynb#top'>**Section 3. Hypothesis Testing**</a> 
  * [1. T-test](#t)
  * [2. ANOVA](#anova)
* <a href='04- DSC 2022 Feature Engineering .ipynb#top'>**Section 4. Feature Engineering**</a> 

Recall that in section 1, we listed out several hypothesis that we would like to test out. In this section, we will focus on how to use statistical methods to do so. 

For hypothesis testing  the first step is always to state clearly a null hypothesis and its alternative hypothesis. The rationale is that if we assumed that the null hypothesis is true, how likely we are to see the data we currently observe, the probability is called p-value. If such p-value is extremely small, then we have some confidence such that our initial null hypothesis is False. Or put it into a more statistical way, we reject the null hypothesis with xx confidendece. 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
cmg = pd.read_excel('cmg.xlsx' ,index_col = 'offeringId')
cmg.head(5)

offeringPricingDate offeringsType  \
offeringId                                                               
b969a1c8-0a26-438a-81e6-5e95f3b30501          2003-10-02           IPO   
1081394b-c9f2-4479-8dd2-528027ff1eea          2005-07-21           IPO   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0          2005-08-04           IPO   
43f06950-8d20-4cfc-b16d-237e0927e1e6          2005-11-10           IPO   
96a13598-121a-41c0-83b5-448843cd8709          2006-02-03           IPO   

                                             offeringsSector  \
offeringId                                                     
b969a1c8-0a26-438a-81e6-5e95f3b30501       Consumer Cyclical   
1081394b-c9f2-4479-8dd2-528027ff1eea  Communication Services   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0  Communication Services   
43f06950-8d20-4cfc-b16d-237e0927e1e6             Industrials   
96a13598-121a-41c0-83b5-448843cd8709                  Energy   

                                                  offeringsSubSector  \
offeringId                                                             
b969a1c8-0a26-438a-81e6-5e95f3b30501                Vehicles & Parts   
1081394b-c9f2-4479-8dd2-528027ff1eea                Telecom Services   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0  Internet Content & Information   
43f06950-8d20-4cfc-b16d-237e0927e1e6             Consulting Services   
96a13598-121a-41c0-83b5-448843cd8709             Oil & Gas Midstream   

                                      offeringsDiscountToLastTrade  \
offeringId                                                           
b969a1c8-0a26-438a-81e6-5e95f3b30501                           0.0   
1081394b-c9f2-4479-8dd2-528027ff1eea                           0.0   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0                           0.0   
43f06950-8d20-4cfc-b16d-237e0927e1e6                           0.0   
96a13598-121a-41c0-83b5-448843cd8709                           0.0   

                                      filingDetailsOfferingPrice issuerCusip  \
offeringId                                                                     
b969a1c8-0a26-438a-81e6-5e95f3b30501                        13.0   501889208   
1081394b-c9f2-4479-8dd2-528027ff1eea                        13.0   209034107   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0                        27.0   056752108   
43f06950-8d20-4cfc-b16d-237e0927e1e6                        16.0   G47567105   
96a13598-121a-41c0-83b5-448843cd8709                        21.0   29273V100   

                                                           issuerName  \
offeringId                                                              
b969a1c8-0a26-438a-81e6-5e95f3b30501        BharCap Acquisition Corp.   
1081394b-c9f2-4479-8dd2-528027ff1eea        GrandSouth Bancorporation   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0  Brand Velocity Acquisition Corp   
43f06950-8d20-4cfc-b16d-237e0927e1e6                     ProLung Inc.   
96a13598-121a-41c0-83b5-448843cd8709     Golden Star Acquisition Corp   

                                      -15SharePrice  -14SharePrice  ...  \
offeringId                                                          ...   
b969a1c8-0a26-438a-81e6-5e95f3b30501            NaN            NaN  ...   
1081394b-c9f2-4479-8dd2-528027ff1eea            NaN            NaN  ...   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0            NaN            NaN  ...   
43f06950-8d20-4cfc-b16d-237e0927e1e6            NaN            NaN  ...   
96a13598-121a-41c0-83b5-448843cd8709            NaN            NaN  ...   

                                      -1SharePrice  \
offeringId                                           
b969a1c8-0a26-438a-81e6-5e95f3b30501           NaN   
1081394b-c9f2-4479-8dd2-528027ff1eea           NaN   
714a166d-9eb0-4b3c-ab8e-7c0dc6f21ee0           NaN   
43f06950-8d20-4cfc-b16d-237e0927e1e6           NaN   
96a13598-121a-41c0-83b5-448843cd8709           NaN   

                                                                           underwriters  \
offe

<a id='t'></a>
## 1. T-test

Hypothesis: if a deal was made at a discount, then the deal would likely to have worse performance in the future. 

The key assumption of a t-test is that under each one of the two levels, the data is noramlly distributed(Gaussian) . Be sure to always check the underlying assumptions before performing any analysis.

According to the boxplot, we observe that for both discount and no-discount group, the distribution of one-day performance is normally distributed. 

In [5]:
cmg['discount'] = cmg['offeringsDiscountToLastTrade']<0
sns.boxplot(x="discount", y="180SharePrice", data=cmg,showfliers = False)

<AxesSubplot:xlabel='discount', ylabel='180SharePrice'>

<a id='anova'></a>
## 2. One Way - Analysis of Variance (ANOVA)

What happens when we have more than two categories for our explanatory variable? Then a t-test would not be enough. In this part, we introduce one way analysis of varaince(one-way ANOVA).

Prior to the hypothesis testing, let's agian first check if the assumption of ANOVA is satisfied. 

In [6]:
temp = cmg.groupby(by = 'offeringsSector')['180SharePrice'].agg(['count', 'mean', 'median', 'var']).reset_index()
temp

offeringsSector  count       mean  median           var
0          Basic Materials    236  26.706637  14.895   1691.767703
1   Communication Services    336  41.666802  16.430  16499.334016
2        Consumer Cyclical    688  33.631770  20.750   2343.978188
3       Consumer Defensive    282  28.367036  20.810   1165.029518
4                   Energy    704  35.057314  24.130   5568.920524
5       Financial Services   1286  23.845112  12.270   1426.662525
6               Healthcare   2301  32.309370  15.020   7739.495303
7              Industrials    680  29.927585  17.765   5496.606006
8              Real Estate    788  33.031774  19.765   2341.923000
9               Technology   1037  34.721705  19.690   5555.365850
10               Utilities    151  36.458987  27.530    619.905306

In [8]:
plt.figure(figsize = (20, 5))
sns.boxplot(x = 'offeringsSector', y = '180SharePrice', data = cmg, showfliers = False)

<AxesSubplot:xlabel='offeringsSector', ylabel='180SharePrice'>

Since the p-value is extremely small(almost 0), which is clearly smaller than our preset cofidence level threshold(alpha = 0.05). Then we shall reject the null hypothesis and claim that sector matters. 

In [10]:
from scipy.stats import f_oneway
performanceBySector = cmg.filter(items = ["offeringsSector","180SharePrice"]).pivot(columns = "offeringsSector", values = "180SharePrice")

In [11]:
F, p = f_oneway(
                performanceBySector['Communication Services'].dropna(),
                performanceBySector['Consumer Cyclical'].dropna(),
                performanceBySector['Consumer Defensive'].dropna(), 
                performanceBySector['Energy'].dropna(), 
                performanceBySector['Financial Services'].dropna(),
                performanceBySector['Healthcare'].dropna(),
                performanceBySector['Industrials'].dropna(),
                performanceBySector['Real Estate'].dropna(),
                performanceBySector['Technology'].dropna(),
                performanceBySector['Utilities'].dropna())
print('F-Statistic=%.3f, p=%.3f' % (F, p))

F-Statistic=3.097, p=0.001


If you are intersted in more in-depth explanations, check out  
https://www.stat.cmu.edu/~hseltman/309/Book/chapter7.pdf for one-way ANOVA
https://www.stat.cmu.edu/~hseltman/309/Book/chapter11.pdf for two-way ANOVA.